# Importações


In [380]:
import geopandas
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv

# Configurações iniciais


In [381]:
load_dotenv()

True

# Leitura dos dados


In [382]:
inventory: pd.DataFrame = pd.read_csv(
    os.environ.get("INVENTORY_FILE_PATH"),
    encoding="ISO-8859-1",
)

In [383]:
inventory.head()

,area,plot,tree,common.name,scientific.name,family.name,DBH,type,Dead,D.class,date,UTM.Easting,UTM.Northing
0,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,20160824,840127.0705,9674180.998
1,DUC_A01,2,001b,uxirana,Sacoglottis guianensis,Humiriaceae,43.6,O,False,NaN,20160824,840098.9678,9674177.868
2,DUC_A01,2,002a,mapatirana,Pourouma minor,Urticaceae,31.5,O,False,NaN,20160824,840126.4377,9674180.705
3,DUC_A01,2,002b,abiurana,Pouteria gongrijpii,Sapotaceae,39.2,O,False,NaN,20160824,840109.4829,9674171.803
4,DUC_A01,2,003a,roxinho,Peltogyne lecointei,Fabaceae,10.0,O,False,NaN,20160824,840125.6847,9674181.313


In [384]:
inventory_plot_location: geopandas.GeoDataFrame = geopandas.read_file(
    os.environ.get("INVENTORY_PLOT_LOCATION_FILE_PATH"),
)

In [385]:
inventory_plot_location.head()

,plot_ID,geometry
0,P20,"POLYGON ((840040.725 9674392.229, 840082.852 9..."
1,P02,"POLYGON ((840077.054 9674170.361, 840124.283 9..."
2,P07,"POLYGON ((840331.289 9673499.875, 840372.258 9..."
3,P09,"POLYGON ((840301.102 9673285.583, 840340.905 9..."
4,P08,"POLYGON ((839205.643 9673316.074, 839256.495 9..."


In [386]:
wood_density_dataframe = pd.read_excel(
    os.environ.get("WOOD_DENSITY_FILE_PATH"),
    sheet_name="Data",
)

In [387]:
wood_density_dataframe.head()

,Number,Family,Binomial,"Wood density (g/cm^3), oven dry mass/fresh volume",Region,Reference Number
0,1,Fabaceae,Abarema jupunba,0.78000,South America (tropical),114
1,2,Fabaceae,Abarema jupunba,0.66000,South America (tropical),198
2,3,Fabaceae,Abarema jupunba,0.55104,South America (tropical),52
3,4,Fabaceae,Abarema jupunba,0.53382,South America (tropical),65
4,5,Fabaceae,Abarema jupunba,0.55104,South America (tropical),189


# Exploração dos dados


In [388]:
inventory["scientific.name"].unique(), len(inventory["scientific.name"].unique())

(array(['Licania guianensis', 'Sacoglottis guianensis', 'Pourouma minor',
        'Pouteria gongrijpii', 'Peltogyne lecointei', nan,
        'Protium paniculatum', 'Qualea paraensis', 'Mouriri collocarpa',
        'Eschweilera ovata', 'Pouteria sp.', 'Ormosia grossa',
        'Ocotea cernua', 'Eperua bijuga', 'Brosimum rubescens',
        'Oenocarpus bataua', 'Hevea brasiliensis', 'Inga thibaudiana',
        'Eriotheca globosa', 'Iryanthera paraensis', 'Dipteryx odorata',
        'Sterculia pruriens', 'Miconia poeppigii', 'Sloanea guianensis',
        'Poeppigia procera', 'Xylopia sp.', 'Neea oppositifolia',
        'Geissospermum sericeum', 'Goupia glabra', 'Eschweilera micrantha',
        'Inga alba', 'Diospyros artanthifolia', 'Eschweilera coriacea',
        'Micrandra minor', 'Trichilia micrantha',
        'Tetragastris panamensis', 'Ormosia coccinea', 'Guarea guidonia',
        'Couepia robusta', 'Pouteria guianensis', 'Ocotea canaliculata',
        'Licania octandra', 'Guatteria 

In [389]:
inventory["family.name"].unique(), len(inventory["family.name"].unique())

(array(['Chrysobalanaceae', 'Humiriaceae', 'Urticaceae', 'Sapotaceae',
        'Fabaceae', nan, 'Burseraceae', 'Vochysiaceae', 'Melastomataceae',
        'Lecythidaceae', 'Lauraceae', 'Moraceae', 'Arecaceae',
        'Euphorbiaceae', 'Malvaceae', 'Myristicaceae', 'Elaeocarpaceae',
        'Annonaceae', 'Nyctaginaceae', 'Apocynaceae', 'Goupiaceae',
        'Ebenaceae', 'Meliaceae', 'Violaceae', 'Olacaceae',
        'Caryocaraceae', 'Combretaceae', 'Loganiaceae', 'Myrtaceae',
        'Simaroubaceae', 'Rubiaceae', 'Bignoniaceae', 'Malpighiaceae',
        'Salicaceae', 'Boraginaceae', 'Anacardiaceae', 'Lamiaceae',
        'Ochnaceae', 'Clusiaceae', 'Calophyllaceae'], dtype=object),
 40)

# Transformações


In [390]:
wood_density_dataframe = wood_density_dataframe.rename(
    {
        "Family": "family.name",
        "Binomial": "scientific.name",
        "Wood density (g/cm^3), oven dry mass/fresh volume": "wood.density",
        "Reference Number": "reference.number",
    },
    axis=1,
)

In [391]:
mean_wood_density_dataframe = (
    wood_density_dataframe.groupby(["family.name", "scientific.name"])["wood.density"]
    .mean("wood.density")
    .reset_index()
)

In [392]:
wood_density_dataframe.head()

,Number,family.name,scientific.name,wood.density,Region,reference.number
0,1,Fabaceae,Abarema jupunba,0.78000,South America (tropical),114
1,2,Fabaceae,Abarema jupunba,0.66000,South America (tropical),198
2,3,Fabaceae,Abarema jupunba,0.55104,South America (tropical),52
3,4,Fabaceae,Abarema jupunba,0.53382,South America (tropical),65
4,5,Fabaceae,Abarema jupunba,0.55104,South America (tropical),189


In [393]:
inventory = inventory.merge(
    mean_wood_density_dataframe,
    on="scientific.name",
    how="left",
)

In [394]:
inventory = inventory.rename(
    {
        "family.name_x": "family.name",
        "Binomial": "scientific.name",
        "wood.density": "family.scientific.wood.density",
        "Reference Number": "reference.number",
    },
    axis=1,
)

In [395]:
inventory.head()

,area,plot,tree,common.name,scientific.name,family.name,DBH,type,Dead,D.class,date,UTM.Easting,UTM.Northing,family.name_y,family.scientific.wood.density
0,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,20160824,840127.0705,9674180.998,NaN,NaN
1,DUC_A01,2,001b,uxirana,Sacoglottis guianensis,Humiriaceae,43.6,O,False,NaN,20160824,840098.9678,9674177.868,Humiriaceae,0.835775
2,DUC_A01,2,002a,mapatirana,Pourouma minor,Urticaceae,31.5,O,False,NaN,20160824,840126.4377,9674180.705,Urticaceae,0.435953
3,DUC_A01,2,002b,abiurana,Pouteria gongrijpii,Sapotaceae,39.2,O,False,NaN,20160824,840109.4829,9674171.803,Sapotaceae,0.799403
4,DUC_A01,2,003a,roxinho,Peltogyne lecointei,Fabaceae,10.0,O,False,NaN,20160824,840125.6847,9674181.313,Fabaceae,0.763333


In [396]:
mean_wood_density_dataframe = (
    wood_density_dataframe.groupby(["family.name"])["wood.density"]
    .mean("wood.density")
    .reset_index()
)

In [397]:
mean_wood_density_dataframe = mean_wood_density_dataframe.rename(
    {
        "wood.density": "family.wood.density",
    },
    axis=1,
)


In [398]:
mean_wood_density_dataframe = mean_wood_density_dataframe.rename(
    {
        "wood.density": "family.wood.density",
    },
    axis=1,
)

mean_wood_density_dataframe.head()

,family.name,family.wood.density
0,Acanthaceae,0.670709
1,Achariaceae,0.605300
2,Actinidiaceae,0.398541
3,Adoxaceae,0.446333
4,Akaniaceae,0.560710


In [399]:
inventory = inventory.merge(mean_wood_density_dataframe, on="family.name", how="left")


# wood_density_dataframe["reference.number"].apply(str)

In [400]:
inventory


,area,plot,tree,common.name,scientific.name,family.name,DBH,type,Dead,D.class,date,UTM.Easting,UTM.Northing,family.name_y,family.scientific.wood.density,family.wood.density
0,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,20160824,840127.0705,9674180.998,NaN,NaN,0.780701
1,DUC_A01,2,001b,uxirana,Sacoglottis guianensis,Humiriaceae,43.6,O,False,NaN,20160824,840098.9678,9674177.868,Humiriaceae,0.835775,0.771500
2,DUC_A01,2,002a,mapatirana,Pourouma minor,Urticaceae,31.5,O,False,NaN,20160824,840126.4377,9674180.705,Urticaceae,0.435953,0.344090
3,DUC_A01,2,002b,abiurana,Pouteria gongrijpii,Sapotaceae,39.2,O,False,NaN,20160824,840109.4829,9674171.803,Sapotaceae,0.799403,0.702085
4,DUC_A01,2,003a,roxinho,Peltogyne lecointei,Fabaceae,10.0,O,False,NaN,20160824,840125.6847,9674181.313,Fabaceae,0.763333,0.678134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1164,DUC_A01,20,048a,quarubarana,Erisma uncinatum,Vochysiaceae,24.0,O,False,NaN,20160830,840072.5767,9674390.969,Vochysiaceae,0.523047,0.551651
1165,DUC_A01,20,049a,fava saboeiro,Abarema jupunba,Fabaceae,14.0,O,False,NaN,20160830,840071.0245,9674395.507,Fabaceae,0.585129,0.678134
1166,DUC_A01,20,050a,mamorana,Eriotheca globosa,Malvaceae,22.0,O,False,NaN,20160830,840069.0191,9674396.896,Malvaceae,0.410000,0.476090
1167,DUC_A01,20,051a,louro cheiroso,Diospyros artanthifolia,Ebenaceae,11.9,O,False,NaN,20160830,840069.1150,9674398.094,NaN,NaN,0.697848


In [401]:
inventory = inventory.drop("family.name_y", axis=1)

In [402]:
inventory = inventory.rename(
    {
        "family.name_x": "family.name",
        "scientific.name_x": "scientific.name"
    },
    axis=1,
)


In [403]:
inventory.head()

,area,plot,tree,common.name,scientific.name,family.name,DBH,type,Dead,D.class,date,UTM.Easting,UTM.Northing,family.scientific.wood.density,family.wood.density
0,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,20160824,840127.0705,9674180.998,NaN,0.780701
1,DUC_A01,2,001b,uxirana,Sacoglottis guianensis,Humiriaceae,43.6,O,False,NaN,20160824,840098.9678,9674177.868,0.835775,0.771500
2,DUC_A01,2,002a,mapatirana,Pourouma minor,Urticaceae,31.5,O,False,NaN,20160824,840126.4377,9674180.705,0.435953,0.344090
3,DUC_A01,2,002b,abiurana,Pouteria gongrijpii,Sapotaceae,39.2,O,False,NaN,20160824,840109.4829,9674171.803,0.799403,0.702085
4,DUC_A01,2,003a,roxinho,Peltogyne lecointei,Fabaceae,10.0,O,False,NaN,20160824,840125.6847,9674181.313,0.763333,0.678134


In [404]:
inventory["wood.density"] = inventory["family.scientific.wood.density"].combine_first(
    inventory["family.wood.density"]
)

In [405]:
inventory

,area,plot,tree,common.name,scientific.name,family.name,DBH,type,Dead,D.class,date,UTM.Easting,UTM.Northing,family.scientific.wood.density,family.wood.density,wood.density
0,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,20160824,840127.0705,9674180.998,NaN,0.780701,0.780701
1,DUC_A01,2,001b,uxirana,Sacoglottis guianensis,Humiriaceae,43.6,O,False,NaN,20160824,840098.9678,9674177.868,0.835775,0.771500,0.835775
2,DUC_A01,2,002a,mapatirana,Pourouma minor,Urticaceae,31.5,O,False,NaN,20160824,840126.4377,9674180.705,0.435953,0.344090,0.435953
3,DUC_A01,2,002b,abiurana,Pouteria gongrijpii,Sapotaceae,39.2,O,False,NaN,20160824,840109.4829,9674171.803,0.799403,0.702085,0.799403
4,DUC_A01,2,003a,roxinho,Peltogyne lecointei,Fabaceae,10.0,O,False,NaN,20160824,840125.6847,9674181.313,0.763333,0.678134,0.763333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1164,DUC_A01,20,048a,quarubarana,Erisma uncinatum,Vochysiaceae,24.0,O,False,NaN,20160830,840072.5767,9674390.969,0.523047,0.551651,0.523047
1165,DUC_A01,20,049a,fava saboeiro,Abarema jupunba,Fabaceae,14.0,O,False,NaN,20160830,840071.0245,9674395.507,0.585129,0.678134,0.585129
1166,DUC_A01,20,050a,mamorana,Eriotheca globosa,Malvaceae,22.0,O,False,NaN,20160830,840069.0191,9674396.896,0.410000,0.476090,0.410000
1167,DUC_A01,20,051a,louro cheiroso,Diospyros artanthifolia,Ebenaceae,11.9,O,False,NaN,20160830,840069.1150,9674398.094,NaN,0.697848,0.697848


In [406]:
inventory[inventory["wood.density"].isnull()]["common.name"].unique()

array(['morta', 'tamaguare', 'ucuuba da varzea', 'carana', 'fava rosa',
       'pincel de macaco', 'breu brrote', 'abacabeira', 'jarana vermelha',
       'taguari', 'tanenbuco amarelo', 'acaizeiro', 'liana',
       'faveira amargosa', 'zaguari', 'virola/ucuuba preta',
       'muiracatiara rajada', 'abiu rosadinha', 'burra leteira',
       'abirana', 'gaivota', 'abirana branca', 'carape', 'mirindiba',
       'pedra ume', 'brteu branco', 'timbora', 'mamominho',
       'ucuuba folha grande', 'tanenbuco', 'faveira rosa',
       'nao identificada', 'guajara vermelho', 'muiracacaco',
       'acariguara', 'acariguarana', 'puriu', ' liana', 'seringuarana',
       'nao indetificada', 'manguirana', 'virola(ucuuba)', 'sorvao',
       'urucurana branca', 'tento cachua', 'ipê amarelo', 'uxi coroa',
       'quariuba', 'mirapiranga', 'ucuuba t.f', 'envira cunacaru',
       'seringarana preta', 'pocoro', 'cacaui', 'urucurana branco'],
      dtype=object)

In [407]:
inventory[inventory["wood.density"].isnull()].count()

area                              183
plot                              183
tree                              183
common.name                       183
scientific.name                     0
family.name                         0
DBH                               182
type                              183
Dead                              183
D.class                            74
date                              183
UTM.Easting                       183
UTM.Northing                      183
family.scientific.wood.density      0
family.wood.density                 0
wood.density                        0
dtype: int64

In [410]:
inventory = inventory.dropna(subset=["wood.density"]).drop(
    ["family.scientific.wood.density", "family.wood.density", "family.name"], axis=1
)

In [411]:
inventory["tree.height"] = 38.1 * (1 - np.exp(-0.0693 * inventory["DBH"] ** 0.826))

In [412]:
inventory

,area,plot,tree,common.name,scientific.name,DBH,type,Dead,D.class,date,UTM.Easting,UTM.Northing,wood.density,tree.height
0,DUC_A01,2,001a,macucu,Licania guianensis,49.6,O,False,NaN,20160824,840127.0705,9674180.998,0.780701,31.430128
1,DUC_A01,2,001b,uxirana,Sacoglottis guianensis,43.6,O,False,NaN,20160824,840098.9678,9674177.868,0.835775,30.146223
2,DUC_A01,2,002a,mapatirana,Pourouma minor,31.5,O,False,NaN,20160824,840126.4377,9674180.705,0.435953,26.597750
3,DUC_A01,2,002b,abiurana,Pouteria gongrijpii,39.2,O,False,NaN,20160824,840109.4829,9674171.803,0.799403,29.025875
4,DUC_A01,2,003a,roxinho,Peltogyne lecointei,10.0,O,False,NaN,20160824,840125.6847,9674181.313,0.763333,14.149619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1164,DUC_A01,20,048a,quarubarana,Erisma uncinatum,24.0,O,False,NaN,20160830,840072.5767,9674390.969,0.523047,23.463946
1165,DUC_A01,20,049a,fava saboeiro,Abarema jupunba,14.0,O,False,NaN,20160830,840071.0245,9674395.507,0.585129,17.459601
1166,DUC_A01,20,050a,mamorana,Eriotheca globosa,22.0,O,False,NaN,20160830,840069.0191,9674396.896,0.410000,22.459927
1167,DUC_A01,20,051a,louro cheiroso,Diospyros artanthifolia,11.9,O,False,NaN,20160830,840069.1150,9674398.094,0.697848,15.807484


In [ ]:
inventory = inventory.merge(inventory.groupby('plot')['tree.height'].mean().reset_index().rename({'tree.height': 'lorey.height'}, axis=1 ), on='plot')

In [ ]:
inventory

,area,plot,tree,common.name,scientific.name,family.name,DBH,type,Dead,D.class,date,UTM.Easting,UTM.Northing,wood.density,scientific.name_y,tree.height,lorey.height
0,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,20160824,840127.0705,9674180.998,0.820000,Acioa edulis,31.430128,23.896686
1,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,20160824,840127.0705,9674180.998,0.860000,Acioa guianensis,31.430128,23.896686
2,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,20160824,840127.0705,9674180.998,0.665000,Atuna racemosa,31.430128,23.896686
3,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,20160824,840127.0705,9674180.998,0.753333,Chrysobalanus icaco,31.430128,23.896686
4,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,20160824,840127.0705,9674180.998,0.842812,Chrysobalanus indet,31.430128,23.896686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276827,DUC_A01,20,052a,louro pimenta,Ocotea canaliculata,Lauraceae,11.2,O,False,NaN,20160830,840070.2963,9674398.520,0.400000,Rhodostemonodaphne praeclara,15.216206,22.676027
276828,DUC_A01,20,052a,louro pimenta,Ocotea canaliculata,Lauraceae,11.2,O,False,NaN,20160830,840070.2963,9674398.520,0.420000,Sassafras albidum,15.216206,22.676027
276829,DUC_A01,20,052a,louro pimenta,Ocotea canaliculata,Lauraceae,11.2,O,False,NaN,20160830,840070.2963,9674398.520,0.469625,Sassafras tzumu,15.216206,22.676027
276830,DUC_A01,20,052a,louro pimenta,Ocotea canaliculata,Lauraceae,11.2,O,False,NaN,20160830,840070.2963,9674398.520,0.552297,Sextonia rubra,15.216206,22.676027


In [ ]:
len(inventory['family.name'].unique())

38